In [1]:
import re
import json
import numpy as np
import pandas as pd

In [3]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
data_folder = './data'
main_dataset = 'compas-scores.csv'
#main_dataset2 = 'compas-scores-raw.csv'

In [397]:
df_init = pd.read_csv (data_folder+'/'+main_dataset)
df = df_init

In [398]:
df.shape

(11757, 47)

In [399]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,-1,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,NaN,13009779CF10A,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,NaN,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,NaN,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [400]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'num_r_cases',
       'r_case_number', 'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out',
       'is_violent_recid', 'num_vr_cases', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date'],
      dtype='object')

### Let's clean information from the future

In [401]:
violence_features = ['num_vr_cases', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date']

df = df.drop(violence_features, axis=1)

In [402]:
recediv_features = ['num_r_cases',
       'r_case_number', 'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out']

df = df.drop(recediv_features, axis=1)

In [403]:
columns_to_drop = ['id', 'last', 'first', 'c_case_number',  'type_of_assessment', 'score_text']

df = df.drop(columns_to_drop, axis=1)

In [404]:
df.head()

,name,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,is_violent_recid,decile_score.1,screening_date
0,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,0,1,2013-08-14
1,michael ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,-1,0,5,2014-12-31
2,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,1,3,2013-01-27
3,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,0,4,2013-04-14
4,marcu brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,0,8,2013-01-13


In [405]:
len(df.columns)

24

### Now we'll filter out our data 

In [406]:
df.dtypes

name                        object
compas_screening_date       object
sex                         object
dob                         object
age                          int64
age_cat                     object
race                        object
juv_fel_count                int64
decile_score                 int64
juv_misd_count               int64
juv_other_count              int64
priors_count                 int64
days_b_screening_arrest    float64
c_jail_in                   object
c_jail_out                  object
c_offense_date              object
c_arrest_date               object
c_days_from_compas         float64
c_charge_degree             object
c_charge_desc               object
is_recid                     int64
is_violent_recid             int64
decile_score.1               int64
screening_date              object
dtype: object

In [407]:
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])

In [408]:
df['time_in_jail'] = round((df['c_jail_out'] - df['c_jail_in']) / np.timedelta64(1, 'D'))

In [409]:
df['time_in_jail'].nlargest(10)

1263     2153.0
1890     1009.0
4758      972.0
10103     881.0
2031      868.0
8498      854.0
1659      800.0
6206      749.0
8469      742.0
6293      716.0
Name: time_in_jail, dtype: float64

In [410]:
df.shape

(11757, 25)

In [411]:
#df[df['time_in_jail'] > 365].shape

In [412]:
df = df[df['c_charge_degree'] != 'O']

In [413]:
df.shape

(11003, 25)

In [414]:
df = df[df['is_recid'] != -1]

In [415]:
df.shape

(11002, 25)

In [416]:
df = df[abs(df['days_b_screening_arrest'])<=30]

In [417]:
df.head()

,name,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,is_violent_recid,decile_score.1,screening_date,time_in_jail
0,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,0,1,2013-08-14,1.0
2,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,1,3,2013-01-27,10.0
3,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,0,4,2013-04-14,1.0
6,marsha miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,0,1,0,0,0,0.0,2013-11-30 04:50:18,2013-12-01 12:28:56,2013-11-30,NaN,0.0,M,Battery,0,0,1,2013-11-30,1.0
7,edward riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,0,6,0,0,14,-1.0,2014-02-18 05:08:24,2014-02-24 12:18:30,2014-02-18,NaN,1.0,F,Possession Burglary Tools,1,0,6,2014-02-19,6.0


### Time for FE

In [418]:
def el_in_str(s, l):
    result = False
    
    for i in l:
        if i.lower() in str(s).lower():
            result = True
    return result

def str_to_cat(x, l, default = 'other'):
    result = default
    
    if x is not None:
        for i in l:
            if el_in_str(x, [i]):
                result = i
    return result

In [419]:
df['juv_count'] = df['juv_fel_count']+df['juv_misd_count']+df['juv_other_count']
df = df.drop(['juv_fel_count','juv_misd_count','juv_other_count'], axis=1)

In [420]:
categories = ['assault', 'battery', 'burglary', 'resisting', 'mischief', 'tampering', 'lewdness', 'poss', 'cannabis', 'driving', 'sexual', 'no charge', 'grand theft']


In [421]:
df['c_cat'] = df['c_charge_desc'].map(lambda x: str_to_cat(x, categories))

In [422]:
df['weapon_firearm'] = df['c_charge_desc'].map(lambda x: el_in_str(x, ['firearm', 'weapon']))

In [423]:
df.head(10)

,name,compas_screening_date,sex,dob,age,age_cat,race,decile_score,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,is_violent_recid,decile_score.1,screening_date,time_in_jail,juv_count,c_cat,weapon_firearm
0,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,1,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,0,1,2013-08-14,1.0,0,assault,True
2,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,3,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,1,3,2013-01-27,10.0,0,battery,False
3,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,4,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,0,4,2013-04-14,1.0,1,poss,False
6,marsha miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,1,0,0.0,2013-11-30 04:50:18,2013-12-01 12:28:56,2013-11-30,NaN,0.0,M,Battery,0,0,1,2013-11-30,1.0,0,battery,False
7,edward riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,6,14,-1.0,2014-02-18 05:08:24,2014-02-24 12:18:30,2014-02-18,NaN,1.0,F,Possession Burglary Tools,1,0,6,2014-02-19,6.0,0,poss,False
8,steven stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,4,3,-1.0,2013-08-29 08:55:23,2013-08-30 08:42:13,NaN,2013-08-29,1.0,F,arrest case no charge,0,0,4,2013-08-30,1.0,0,no charge,False
9,elizabeth thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,1,0,-1.0,2014-03-15 05:35:34,2014-03-18 04:28:46,2014-03-15,NaN,1.0,M,Battery,0,0,1,2014-03-16,3.0,0,battery,False
10,darrell blackburn,2013-12-09,Male,1995-11-16,20,Less than 25,Caucasian,10,0,-1.0,2013-12-08 08:17:29,2014-01-10 12:16:01,2013-12-08,NaN,1.0,F,Possession Of Alprazolam,0,0,10,2013-12-09,33.0,1,poss,False
11,jamie good,2014-09-05,Female,1989-04-23,26,25 - 45,Caucasian,5,0,-1.0,2014-09-04 02:37:17,2014-09-05 08:41:35,2014-09-04,NaN,1.0,F,Possession Of Alprazolam,0,0,5,2014-09-05,1.0,0,poss,False
13,benjamin franc,2013-11-26,Male,1988-06-01,27,25 - 45,Caucasian,4,0,-1.0,2013-11-25 06:31:06,2013-11-26 08:26:57,2013-11-25,NaN,1.0,F,"Poss 3,4 MDMA (Ecstasy)",0,0,4,2013-11-26,1.0,0,poss,False


In [424]:
drop_first = True

In [425]:
feat2dummie = ['sex','age_cat','race','c_charge_degree','c_cat','weapon_firearm']

In [426]:
for i in feat2dummie:
    one_hot = pd.get_dummies(df[i], prefix=i, drop_first=drop_first)
    df = df.drop(i,axis = 1)
    df = df.join(one_hot)

In [427]:
columns_to_drop = ['name', 'compas_screening_date', 'dob', 'c_jail_in', 'c_jail_out', 'c_offense_date', 'c_arrest_date', 'c_charge_desc', 'decile_score.1', 'time_in_jail', 'screening_date']
df = df.drop(columns_to_drop, axis=1)

### Model training

In [428]:
is_recid, is_violent_recid = df['is_recid'], df['is_violent_recid']

In [429]:
columns_to_drop = ['is_recid', 'is_violent_recid']

df = df.drop(columns_to_drop, axis=1)

In [430]:
df.head(10)

,age,decile_score,priors_count,days_b_screening_arrest,c_days_from_compas,juv_count,sex_Male,age_cat_Greater than 45,age_cat_Less than 25,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_M,c_cat_battery,c_cat_burglary,c_cat_cannabis,c_cat_driving,c_cat_grand theft,c_cat_lewdness,c_cat_mischief,c_cat_no charge,c_cat_other,c_cat_poss,c_cat_resisting,c_cat_sexual,c_cat_tampering,weapon_firearm_True
0,69,1,0,-1.0,1.0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,34,3,0,-1.0,1.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,24,4,4,-1.0,1.0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,44,1,0,0.0,0.0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,41,6,14,-1.0,1.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8,43,4,3,-1.0,1.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,39,1,0,-1.0,1.0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,20,10,0,-1.0,1.0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
11,26,5,0,-1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
13,27,4,0,-1.0,1.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [456]:
dft = df.copy()#df.drop('c_days_from_compas', axis=1)
cols_to_pow = ['age','decile_score','priors_count','juv_count']

max_pow = 3
for k,i in enumerate(cols_to_pow):
    for j in range(1, max_pow):
        dft[i+str(j+1)] = pow(df[i], j+1)
    

In [457]:
#dft = df.drop(['decile_score','days_b_screening_arrest'], axis=1)

In [458]:
train, test, train_y, test_y = train_test_split(dft,
                                                is_recid,
                                                test_size = 0.20,
                                                random_state = 42)

In [515]:
rfc =  RandomForestClassifier(n_estimators=100,
                               oob_score=True,
                               random_state=42)
rfc.fit(train, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [516]:
pred = rfc.predict(test)
#print ("AUC-ROC (oob) = ", roc_auc_score(train_y, model.oob_prediction_))
print ("AUC-ROC (test) = ", roc_auc_score(test_y, pred))

AUC-ROC (test) =  0.636045472092976


#### Now we'll tune parameters

In [467]:
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rfc.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': 1,
 'oob_score': True,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [477]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [200] #[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200]}


In [478]:
rfc_new = RandomForestClassifier()

rfcCV = RandomizedSearchCV(estimator = rfc_new, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rfcCV.fit(train, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=False 
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=False 
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=False 
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=False 
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=False 
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=50, bootstrap=True 
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=50, bootstrap=True 
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=50, bootstrap=True 
[CV]  n_estimators

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.8s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=False, total=   2.8s
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=False, total=   2.8s
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=False, total=   3.0s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=40, bootstrap=False, total=   2.9s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=True 
[CV]  n_estimators=200

[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=True, total=   2.1s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=None, bootstrap=True, total=   2.5s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=None, bootstrap=True, total=   2.6s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=None, bootstrap=True, total=   2.6s
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min

[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   2.1s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=50, bootstrap=True, total=   3.2s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=False, total=   3.5s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False, total=   2.4s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_sa

...

[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=70, bootstrap=True, total=   2.0s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=70, bootstrap=True, total=   2.0s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=20, bootstrap=True, total=   2.1s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True, total=   1.8s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_sam

[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   2.7s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   2.4s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   2.4s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total=   2.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=40, bootstrap=False 
[CV]  n_estimators=200, mi

[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=10, bootstrap=True, total=   1.7s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=10, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=10, bootstrap=True, total=   1.7s
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=False, total=   4.0s
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=False, total=   4.0s
[CV] n_estimators=200, min_sam

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.2min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [486]:
rfcCV.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 200}

In [508]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [200] #[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 15, num = 5)]
max_depth.append(None)
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [10, 20, 30]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 8, 12]
# Method of selecting samples for training each tree
bootstrap = [True]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True],
 'max_depth': [1, 4, 8, 11, 15, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [4, 8, 12],
 'min_samples_split': [10, 20, 30],
 'n_estimators': [200]}


In [509]:
rfc_new = RandomForestClassifier()

rfcCV = RandomizedSearchCV(estimator = rfc_new, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rfcCV.fit(train, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=4, max_features=sqrt, max_depth=1, bootstrap=True 
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=4, max_features=sqrt, max_depth=1, bootstrap=True 
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=4, max_features=sqrt, max_depth=1, bootstrap=True 
[CV]  n_estimators=200, min_

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.1s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=8, max_features=sqrt, max_depth=4, bootstrap=True, total=   1.3s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=8, max_features=sqrt, max_depth=11, bootstrap=True, total=   1.8s
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=8, max_features=sqrt, max_depth=11, bootstrap=True, total=   1.9s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=8, max_features=sqrt, max_depth=11, bootstrap=True, total=   1.8s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=True 
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_sampl

[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=1, bootstrap=True, total=   1.0s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=12, max_features=auto, max_depth=15, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=1, bootstrap=True, total=   0.9s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=auto, max_depth=15, bootstrap=True, total=   1.9s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=auto, max_depth=15, bootstrap=True, total=   2.0s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=4, bootstrap=True 
[CV]  n_estimators=200, min_sa

[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=None, bootstrap=True, total=   2.0s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=8, max_features=sqrt, max_depth=1, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=None, bootstrap=True, total=   1.9s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=8, max_features=sqrt, max_depth=1, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=12, max_features=auto, max_depth=8, bootstrap=True, total=   1.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=12, max_features=auto, max_depth=8, bootstrap=True, total=   1.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, mi

...

[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=sqrt, max_depth=4, bootstrap=True, total=   1.2s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=11, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=4, max_features=sqrt, max_depth=None, bootstrap=True, total=   2.0s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=11, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=4, max_features=sqrt, max_depth=None, bootstrap=True, total=   2.1s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=auto, max_depth=11, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=4, max_features=sqrt, max_depth=None, bootstrap=True, total=   2.1s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=11, bootstrap=True 
[CV]  n_estimators=200,

[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=8, bootstrap=True 
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.7s
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.7s
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.8s
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=12, max_features=sqrt, max_depth=8, bootstrap=True 
[CV]  n_estimators=200, min_sample

[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.5s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=sqrt, max_depth=15, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.4s
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=12, max_features=auto, max_depth=None, bootstrap=True, total=   1.4s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=sqrt, max_depth=15, bootstrap=True 
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=sqrt, max_depth=15, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=30, min_samples_leaf=4, max_features=auto, max_depth=15, bootstrap=True, total=   1.5s
[CV] n_estimators=200, min_samples_split=30, min_samples_leaf=8, max_features=sqrt, max_depth=15, bootstrap=True 
[CV]  n_estimators=

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.8min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200], 'max_features': ['auto', 'sqrt'], 'max_depth': [1, 4, 8, 11, 15, None], 'min_samples_split': [10, 20, 30], 'min_samples_leaf': [4, 8, 12], 'bootstrap': [True]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [510]:
rfcCV.best_params_

{'bootstrap': True,
 'max_depth': 8,
 'max_features': 'sqrt',
 'min_samples_leaf': 8,
 'min_samples_split': 10,
 'n_estimators': 200}

In [513]:
rfc2 =  RandomForestClassifier(n_estimators=500,
                               oob_score=True,
                               random_state=42,
                             bootstrap=rfcCV.best_params_['bootstrap'],
                             max_depth=rfcCV.best_params_['max_depth'],
                             max_features=rfcCV.best_params_['max_features'],
                             min_samples_leaf=rfcCV.best_params_['min_samples_leaf'],
                             min_samples_split=rfcCV.best_params_['min_samples_split'])
rfc2.fit(train, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=8, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [514]:
pred = rfc2.predict(test)
#print ("AUC-ROC (oob) = ", roc_auc_score(train_y, model.oob_prediction_))
print ("AUC-ROC (test) = ", roc_auc_score(test_y, pred))

AUC-ROC (test) =  0.6309075116321596


## The best model was with default parameters

In [517]:
rfc =  RandomForestClassifier(n_estimators=100,
                               oob_score=True,
                               random_state=42)
rfc.fit(train, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [518]:
pred = rfc.predict(test)
#print ("AUC-ROC (oob) = ", roc_auc_score(train_y, model.oob_prediction_))
print ("AUC-ROC (test) = ", roc_auc_score(test_y, pred))

AUC-ROC (test) =  0.636045472092976


### For future work: result interpretability

In [49]:
model.estimators_[3]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=1857819720, splitter='best')

In [34]:
#!pip install treeinterpreter

In [ ]:
#It's working for 

In [41]:
from treeinterpreter import treeinterpreter as ti

for i,row in test.iterrows():

    data_point = pd.DataFrame([row])
    data_point.set_axis(['value_variable'], inplace=True) # Once transposed, it will be the column name
    prediction, bias, contributions = ti.predict(model, data_point)
    
    local_interpretation = data_point.append(
        pd.DataFrame([[round(c[1],3) for c in contributions[0]]], columns=data_point.columns.tolist(), index=['contribution_variable'])
    ).T.sort_values('contribution_variable', ascending=False)
# print(local_interpretation)

In [45]:
print(local_interpretation)

                 value_variable  contribution_variable
age                        25.0                  0.142
race_3                      0.0                  0.011
sex_0                       0.0                  0.011
sex_1                       1.0                  0.011
race_0                      1.0                  0.010
race_1                      0.0                  0.005
race_4                      0.0                  0.001
race_5                      0.0                  0.000
c_charge_degree             0.0                 -0.002
race_2                      0.0                 -0.046
priors_count                1.0                 -0.081


In [47]:
from sklearn import tree

In [ ]:
dot_data = tree.export_graphviz(model,
                                out_file=None,
                                filled=True,
                                rounded=True)

In [ ]:

graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')

In [38]:
data_point

,age,priors_count,c_charge_degree,race_0,race_1,race_2,race_3,race_4,race_5,sex_0,sex_1
value_variable,34,8,0,0,0,1,0,0,0,0,1


In [ ]:
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = df.feature_names,
                class_names = df.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')